# Introduction #

Run the cell below to set everything up.

In [ ]:
# Setup feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.computer_vision.ex3 import *

# Apply Transformations

Run the following cell to get started on this exercise.

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt

img_path = './images/car_feature.jpg'
img = tf.io.read_file(img_path)
img = tf.io.decode_jpeg(img)

plt.figure(figsize=(6, 6))
plt.imshow(tf.squeeze(img), cmap='gray')
plt.axis('off')
plt.show();

### 1) Define Kernel

First define a kernel. You have your choice of what kind of kernel to apply, but here are some ideas:

In [ ]:
kernel = tf.constant([
    # YOUR CODE HERE
    ____
])

# Reformat for tensorflow
kernel = tf.reshape(kernel, [*kernel.shape, 1, 1])

q_1.check()

### 2) Apply Convolution

In [ ]:
# YOUR CODE HERE
image_filter = ____

q_2.check()

Run the next cell to see the result.

In [ ]:
plt.imshow(
    # Reformat for plotting
    tf.squeeze(image_filter)
)
plt.axis('off')
plt.show();

### 3) Apply ReLU

In [ ]:
# YOUR CODE HERE
image_detect = ___

q_3.check()

Run the next cell to see the result.

In [ ]:
plt.imshow(
    # Reformat for plotting
    tf.squeeze(image_detect)
)
plt.axis('off')
plt.show();

### 4) Apply Pooling

In [ ]:
# YOUR CODE HERE
image_condense = ____

q_4.check()

Run the next cell to see the result.

In [ ]:
plt.imshow(
    # Reformat for plotting
    tf.squeeze(image_detect)
)
plt.axis('off')
plt.show();

### 5) Explore Filters

Use the `show_extraction` function from the `visiontools` module to explore the effect of some other filters. Try one of the ones above or invent your own!

In [ ]:
image = np.array(Image.open('./images/car_feature.png'))

kernel = tf.constant([
    # YOUR CODE HERE
    ____
])

visiontools.show_extraction(image, kernel)
q_5.check()

You're not limited to the 3x3 kernels we've seen so far. You could try kernels with other dimensions, like 2x2, 6x6, or even 3x5!

# 6) Explore Feature Maps

Use the `show_feature_maps` function from the `visiontools` module to see the feature maps a model produces. Let's start with VGG16. Run the cell below to make a list of the layers you can visualize.

In [ ]:
model = tf.keras.applications.VGG16(include_top=False, input_shape=img.shape)

layer_names = [layer.name for layer in model.layers
               if layer.__class__.__name__ is 'Conv2D']
print(layer_names)

Now choose a layer from the first block.

In [ ]:
# YOUR CODE HERE
layer_b1 = ____
q_6.a.check()

In [ ]:
q_6.solution()

And once you've gotten a solution, run the code below to produce the visualization.

In [ ]:
visiontools.show_feature_maps(img, layer_name=layer_b1)

Now do the same for a layer from the fourth block

In [ ]:
# YOUR CODE HERE
layer_b4 = ____
q_6.b.check()

In [ ]:
q_6.b.solution()

And once you've gotten a solution, run the code below to produce the visualization.

In [ ]:
visiontools.show_feature_maps(img, layer_name=layer_b4)

Thought question.

In [ ]:
q_6.c.solution()